#### Tensorboard startup

In [1]:
# Load Tensorboard network extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

2023-12-11 13:57:27.801378: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 13:57:27.828678: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 13:57:27.828705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 13:57:27.829389: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 13:57:27.834498: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# clear logs
!rm -rf ./logs/

#### Load Mnist and make model

In [4]:
from tensorflow.keras.layers import Flatten, Dense, Dropout

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

def create_model():
    return tf.keras.models.Sequential([
        Flatten(input_shape=(28,28), name='layers_flatten'),
        Dense(512, activation='relu', name='layers_dense'),
        Dropout(0.2, name='layers_dropout'),
        Dense(10, activation='softmax', name='out_layer')
    ])

#### Model Fitting with Tensorboard

In [6]:
model = create_model()
model.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

logs_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir, histogram_freq=1)

model.fit(
    x=x_train,
    y=y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    callbacks = [tensorboard_callback]
)

2023-12-11 14:08:27.141782: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/5


2023-12-11 14:08:28.172278: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2e020e8c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-11 14:08:28.172309: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti Laptop GPU, Compute Capability 8.6
2023-12-11 14:08:28.176281: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-11 14:08:28.188778: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1702332508.252549   20983 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 8s 4ms/step - loss: 0.2204 - accuracy: 0.9346 - val_loss: 0.1032 - val_accuracy: 0.9685
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0970 - accuracy: 0.9708 - val_loss: 0.0775 - val_accuracy: 0.9749
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0689 - accuracy: 0.9778 - val_loss: 0.0781 - val_accuracy: 0.9743
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0536 - accuracy: 0.9832 - val_loss: 0.0659 - val_accuracy: 0.9807
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0439 - accuracy: 0.9855 - val_loss: 0.0647 - val_accuracy: 0.9816


In [17]:
# Bind to a localhost port. Makes it easier. Had trouble visualizing with VSCode but this worked just fine
%tensorboard --logdir logs/fit20231211-140826 --host localhost --port 8888

Reusing TensorBoard on port 8888 (pid 21935), started 0:20:40 ago. (Use '!kill 21935' to kill it.)

In [19]:
!lsof -i:8888

In [15]:
!kill -9 21935